In [139]:
import keras as k
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering

In [140]:
#read in service data
service_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/service_cost.csv')
service_data.tail()

parts_data = pd.read_csv('/Users/skylerwilson/Desktop/Lighthouse_Labs/Projects/final_project/data/Project_Data/cleaned_parts_data.csv')
parts_data = parts_data.drop(columns=['Unnamed: 0'])

In [141]:
index_to_drop_ = 2381
service_data.drop(index_to_drop_, axis=0, inplace=True)
index_to_drop = 2382
service_data.drop(index_to_drop, axis=0, inplace=True)


In [142]:
def clean_and_convert_columns(df, columns, target_dtype):
    """
    Clean and convert specified columns in a DataFrame to the target data type.

    Args:
        df (pd.DataFrame): DataFrame to clean and convert columns.
        columns (list of str): List of column names to clean and convert.
        target_dtype (type): Target data type to convert columns to.

    Returns:
        pd.DataFrame: Cleaned DataFrame with specified columns converted.
    """
    for column in columns:
        df[column] = df[column].str.replace(',', '')
        df[column] = df[column].astype(target_dtype)
    return df

# Columns to clean and convert
columns_to_clean = service_data.columns  # Replace with the actual column names

# Clean and convert columns to float
service_data = clean_and_convert_columns(service_data, columns_to_clean, float)

In [143]:
service_data.dtypes

Parts Cost            float64
Labor Cost            float64
Sublet Cost           float64
Total Service Cost    float64
dtype: object

In [144]:
avg_service_cost = service_data['Total Service Cost'].mean()
print(f'Average Service Cost: ${round(avg_service_cost, 2)}')

Average Service Cost: $322.86


In [145]:
time_period = ['Sales - Jan', 'Sales - Feb',
       'Sales - Mar', 'Sales - Apr', 'Sales - May', 'Sales - Jun',
       'Sales - Jul', 'Sales - Aug', 'Sales - Sep', 'Sales - Oct',
       'Sales - Nov', 'Sales - Dec', 'Sales - 1st Qtr', 'Sales - 2nd Qtr',
       'Sales - 3rd Qtr', 'Sales - 4th Qtr', 'Sales - Last Year']

In [146]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def find_best_alpha(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    alpha_values = np.linspace(0.01, 0.99, num=20)

    best_alpha = None
    best_mse = float('inf')

    for alpha in alpha_values:
        model = ExponentialSmoothing(y_train, seasonal='add', seasonal_periods=12, trend='add')
        fit_model = model.fit(smoothing_level=alpha)
        forecast = fit_model.forecast(steps=len(X_test))

        mse = mean_squared_error(y_test, forecast)
        if mse < best_mse:
            best_mse = mse
            best_alpha = alpha

    return best_alpha, best_mse

# Replace 'time_period' with the actual column name for your X data
time_period = 'Sales - Last Year'  # Change this to your desired column
X = parts_data[time_period]
y = parts_data['Sales - This Year']

best_alpha, best_mse = find_best_alpha(X, y)
print(f"Best Alpha: {best_alpha}, Best MSE: {best_mse}")


/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(da

Best Alpha: 0.3194736842105263, Best MSE: 153.48201683071352


EOQ = sqrt(2*demand*ordering cost / carrying cost)

demand = use exponential smoothing to forecast demand Ft = a*Xt = (1-a)*Ft-1

ordering cost = frequency * (cost of the part + cost of transportation + lead time costs)

carrying cost = storage cost + capital cost + obsolecence + depreciation + opportunity cost

In [151]:
def calculate_demand(sales_data, alpha):
    demand = sales_data
    initial_forecast = sales_data * 1.15
    forecast = alpha * demand + (1 - alpha) * initial_forecast

    return forecast

period = 'Sales - This Year'
alpha = 0.3194736842105263
parts_data['Demand'] = calculate_demand(parts_data[period], alpha)


In [152]:
parts_data.head()

,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,...,Gross Profit - Sep,Gross Profit - Oct,Gross Profit - Nov,Gross Profit - Dec,COGS,Starting Inventory,Ending Inventory,Average Inventory,Turnover,Demand
0,0,PM1,return spring,triumph,100.0,7.26,7.26,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0 580 463 999,ACC,euro moto internationalrelectrics bosch,euro moto internationalrelectrics,100.0,629.99,629.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00-0450,ACC,"screw, gearbox,inspec&",moto international,100.0,1.95,1.95,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00-548588,ATA,tube 150/90- 15 centre90' stem,butler maps inc,100.0,19.99,19.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00-F01740401,ACC,turn signal light control,moto international,100.0,166.99,166.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
def calculate_ordering_costs(order_quantity, fixed_ordering_costs, carrying_cost_rate, avg_lead_time_days, sales_data):
    lead_time_demand = (sales_data / len(sales_data)) * avg_lead_time_days
    carrying_costs_lead_time = 0.5 * carrying_cost_rate * lead_time_demand  

    total_ordering_costs = fixed_ordering_costs + carrying_costs_lead_time * order_quantity
    
    return total_ordering_costs

order_quantity = parts_data['Quantity Received YTD'].sum()
fixed_ordering_cost = parts_data['Total Cost']* 0.3
carrying_cost_rate = 0.20  
avg_lead_time_days = 14  
sales_data = parts_data['Sales - This Year'] 

# Calculate total ordering costs considering lead time
parts_data['Ordering Cost'] = calculate_ordering_costs(order_quantity, fixed_ordering_cost, carrying_cost_rate, avg_lead_time_days, sales_data)



In [159]:
def calculate_carrying_cost(average_inventory_level, carrying_cost_rate, avg_time_in_inventory):
    """
    Calculate carrying cost for an inventory item.
    
    Parameters:
    average_inventory_level (float): Average inventory level in units.
    carrying_cost_rate (float): Carrying cost rate as a decimal.
    
    Returns:
    float: Carrying cost for the inventory item.
    """
    carrying_cost = average_inventory_level * carrying_cost_rate * avg_time_in_inventory

    return carrying_cost

# Example usage
average_inventory_level = parts_data['Average Inventory']
carrying_cost_rate = 0.125
avg_time_in_inventory = parts_data['Months No Sale']

parts_data['Carrying Cost'] = calculate_carrying_cost(average_inventory_level, carrying_cost_rate, avg_time_in_inventory)


In [160]:
parts_data.head()

,Part Number,Category,Description,Supplier Name,Margin %,Margin,Price,Quantity,Sales Last Month,Sales Last 3 Months,...,Gross Profit - Nov,Gross Profit - Dec,COGS,Starting Inventory,Ending Inventory,Average Inventory,Turnover,Demand,Ordering Cost,Carrying Cost
0,0,PM1,return spring,triumph,100.0,7.26,7.26,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0 580 463 999,ACC,euro moto internationalrelectrics bosch,euro moto internationalrelectrics,100.0,629.99,629.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00-0450,ACC,"screw, gearbox,inspec&",moto international,100.0,1.95,1.95,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00-548588,ATA,tube 150/90- 15 centre90' stem,butler maps inc,100.0,19.99,19.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00-F01740401,ACC,turn signal light control,moto international,100.0,166.99,166.99,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [161]:
parts_data[['Demand', 'Ordering Cost', 'Carrying Cost']].value_counts()

Demand      Ordering Cost  Carrying Cost
0.000000    0.000000       0.000000         83755
1.102079    6.532796       0.000000           479
2.204158    13.065591      0.000000           188
3.306237    19.598387      0.000000            57
4.408316    26.131183      0.000000            47
                                            ...  
0.000000    25.764000      101.465000           1
            25.776000      32.247500            1
            25.803000      0.000000             1
            25.851000      95.931250            1
829.865447  4919.195173    3212.015625          1
Length: 14628, dtype: int64

In [169]:
import math

def EOQ(dataframe, demand_col, ordering_cost_col, carrying_cost_col):
    dataframe['EOQ'] = dataframe.apply(
        lambda row: math.sqrt((2 * row[demand_col] * row[ordering_cost_col]) / row[carrying_cost_col])
                     if row[carrying_cost_col] > 0 else row[demand_col] * math.sqrt(2 * row[ordering_cost_col]),
        axis=1
    )
    return dataframe

demand_col = 'Demand'
ordering_cost_col = 'Ordering Cost'
carrying_cost_col = 'Carrying Cost'

parts_data = EOQ(parts_data, demand_col, ordering_cost_col, carrying_cost_col)

In [ ]:
def custom_loss_function(y_true, y_pred):
    #define the function here:
    # customer LTV: avg_service_cost + avg(add ons cost) + avg(cost of tires) + avg(cost to fix drops)* % likelihood of w_turnover * normalized_turnover + w_eoq * normalized_eoq
    #Obsolecence cost: 95% * (cost per unit)
    
    squared_difference = tf.square(y_true - y_pred)
    mean_squared_error = tf.reduce_mean(squared_difference)

    return mean_squared_error

In [ ]:
def custom_distance_function(x, y):
    #create custom distance function here
    return None